`Last edit on Mar 18 2024 with QuantumClifford 0.9.0 and Julia 1.10.2`

In [4]:
using QuantumClifford
using AbstractAlgebra
using LaTeXStrings
using Quantikz: displaycircuit
using Plots      # Makie is a good alternative plotting library

ArgumentError: ArgumentError: Package AbstractAlgebra not found in current path.
- Run `import Pkg; Pkg.add("AbstractAlgebra")` to install the AbstractAlgebra package.

# States of interest

All of the states will be expressed in one of the many canonical forms available, so that comparisons can be performed easily.

A typical Bell pair

In [5]:
good_bell_state = S"XX
                    ZZ"
canonicalize_rref!(good_bell_state)[1]

+ ZZ
+ XX

A typical GHZ state

In [6]:
good_ghz3_state = S"XXX
                    ZZI
                    IZZ"
canonicalize_rref!(good_ghz3_state)[1]

+ _ZZ
+ Z_Z
+ XXX

The bi-colored graph state version of a GHZ state

In [7]:
good_ghz3_state = S"ZXX
                    XZI
                    XIZ"
canonicalize_rref!(good_ghz3_state)[1]

+ _ZZ
+ ZXX
+ X_Z

# Warmup example: Purifying a Bell pair

We will first run a typical purification circuit on two perfect Bell pairs. Given that all gates and initial states are perfect, the circuit is redundant, but running this calculation is a good sanity check. After all, if the circuit is not preserving perfect Bell pairs, there is little chance it would be doing any purification either.

There are three components to the circuit: the bilateral CPHASE (or CNOT) gates, the coincidence measurement performed by Alice and Bob to check for errors, and a "verify" step at the end of the simulation. Experimentally, Alice and Bob do not have access to this operation, but we add it in our calculation in order to verify whether the final state obtained by this procedure is indeed the state we want.

In terms of library features, we use the `SparseGate` and `BellMeasurement` objects for the "real" part of the circuit and the `VerifyOp` for the verification step.

In [8]:
initial_state = MixedDestabilizer(good_bell_state⊗good_bell_state)
circuit = [
    sCPHASE(1,3),
    sCPHASE(2,4),
    BellMeasurement([sMX(3),sMX(4)]),
    VerifyOp(good_bell_state, [1,2])
]

4-element Vector{QuantumClifford.AbstractOperation}:
 sCPHASE(1,3)
 sCPHASE(2,4)
 BellMeasurement(Union{sMX, sMY, sMZ}[sMX(3, 0), sMX(4, 0)], false)
 VerifyOp(Stabilizer 2×2, [1, 2])

We can run a Monte Carlo simulation with `mctrajectories`, or a Perturbative Expansion calculation with `petrajectories`. Given that there is no noise source in this circuit, all Monte Carlo samples will give the same result.

In [9]:
petrajectories(initial_state, circuit)

Dict{CircuitStatus, Float64} with 3 entries:
  failure:CircuitStatus(3)       => 0.0
  false_success:CircuitStatus(2) => 0.0
  true_success:CircuitStatus(1)  => 1.0

In [10]:
mctrajectories(initial_state, circuit, trajectories=100)

Dict{CircuitStatus, Float64} with 4 entries:
  failure:CircuitStatus(3)       => 0.0
  continue:CircuitStatus(0)      => 0.0
  false_success:CircuitStatus(2) => 0.0
  true_success:CircuitStatus(1)  => 100.0

The possible final statuses (`:undetected_failure`, `:detected_failure`, `:true_success`) come from the definitions of the various circuit operations. `SparseGate` always generates `:continue`, which tells the simulator to continue to the next operation. `BellMeasurement` can report `:detected_failure` or `:continue`, depending whether there was an anti-coincidence or a coincidence. Finally `VerifyOp` reports either `:true_success` if we indeed obtained the desired state, or a `:undetected_failure` if the obtained state is not the desired one even though the measurement step had reported `:continue`. Custom statuses can be implemented as well, as seen later on.

Later on we will also need to compare these results against no purification so we define this circuit for convenience:

In [11]:
nopurification_circuit = [VerifyOp(good_bell_state, [1,2])];

## Define a network noise parameter to be used for the rest of the notebook

For a more interesting calculation, we will now introduce network noise with the `NoiseOpAll` operation, which causes depolarization of certain strength to all of the qubits.

In [12]:
netnoise_value = 0.10
netnoise = UnbiasedUncorrelatedNoise(netnoise_value)
netnoise_opall = NoiseOpAll(netnoise);

In [13]:
[netnoise_opall,circuit...] # The explosions denote depolarization noise

5-element Vector{QuantumClifford.AbstractOperation}:
 NoiseOpAll(UnbiasedUncorrelatedNoise{Float64}(0.1))
 sCPHASE(1,3)
 sCPHASE(2,4)
 BellMeasurement(Union{sMX, sMY, sMZ}[sMX(3, 0), sMX(4, 0)], false)
 VerifyOp(Stabilizer 2×2, [1, 2])

In [14]:
pe_netnoise = petrajectories(initial_state, [netnoise_opall,circuit...])

Dict{CircuitStatus, Float64} with 3 entries:
  failure:CircuitStatus(3)       => 0.1944
  false_success:CircuitStatus(2) => 0.0972
  true_success:CircuitStatus(1)  => 0.6561

In [15]:
pe_netnoise_nopurification = petrajectories(good_bell_state,
                                            [netnoise_opall,nopurification_circuit...],max_order=2)

Dict{CircuitStatus, Float64} with 3 entries:
  failure:CircuitStatus(3)       => 0.0
  false_success:CircuitStatus(2) => 0.186667
  true_success:CircuitStatus(1)  => 0.813333

In [16]:
latexstring("""
    Therefore a network depolarization rate of $(netnoise_value)
    causes the fidelity to drop to
    \$F_{0}=
    $(round(pe_netnoise_nopurification[true_success_stat],digits=3))
    \$.
    """)

UndefVarError: UndefVarError: `latexstring` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [17]:
latexstring("""
    But purification can increase the fidelity to
    \$\\frac{\\mathrm{true\\ success}}{\\mathrm{true\\ success}+\\mathrm{undetected\\ failure}}=
    $(round(pe_netnoise[true_success_stat] / (pe_netnoise[true_success_stat]+pe_netnoise[false_success_stat]),digits=3))
    \$.
    """)

UndefVarError: UndefVarError: `latexstring` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [18]:
latexstring("""
    With a rate of positive coincidence measurements of
    \$\\mathrm{true\\ success}+\\mathrm{undetected\\ failure}=
    $(round(pe_netnoise[true_success_stat]+pe_netnoise[failure_stat],digits=3))
    \$.
    """)

UndefVarError: UndefVarError: `latexstring` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

### Monte Carlo instead of Perturbative Expansions

The 10% network error rate we have picked is not all that small, and the perturbative expansion will not be all that accurate. We can attempt Monte Carlo simulations. For very large circuits this would be cheaper than perturbative expansions. Choosing the best method is a balancing act that depends on the desired accuracy, the size of the circuit, and the number of noisy components.

In [19]:
mc_netnoise = mctrajectories(initial_state, [netnoise_opall,circuit...],
                             trajectories=10000)

Dict{CircuitStatus, Float64} with 4 entries:
  failure:CircuitStatus(3)       => 2219.0
  continue:CircuitStatus(0)      => 0.0
  false_success:CircuitStatus(2) => 1182.0
  true_success:CircuitStatus(1)  => 6599.0

### Symbolic Perturbative Expansions

The perturbative expansions can be done with any of the available symbolic libraries under Julia (including the Julia interface to the SymPy Python library). Here we give an example with `AbstractAlgebra.jl`, part of Nemo. Fractions are a bit clunky, so we treat the numerator and denominator separately. `SymPy` would be simpler to use, but it can be a bit slower.

We will define two symbols, one for the network noise, and one for the gate noise to be used later.

In [20]:
using AbstractAlgebra
field = RealField # If you want to use multiprecision floats
# field = QQ # If you want to use rationals
R, (e_netn, e_gate) = polynomial_ring(field, ["e_n", "e_g"])
sym_unity = R(1);

ArgumentError: ArgumentError: Package AbstractAlgebra not found in current path.
- Run `import Pkg; Pkg.add("AbstractAlgebra")` to install the AbstractAlgebra package.

In [21]:
sym_netnoise = UnbiasedUncorrelatedNoise(e_netn)
sym_netnoise_opall = NoiseOpAll(sym_netnoise);

UndefVarError: UndefVarError: `e_netn` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [22]:
sym_pe_netnoise = petrajectories(initial_state, [sym_netnoise_opall,circuit...],
                             branch_weight=sym_unity)

sym_true_success = sym_pe_netnoise[true_success_stat]
sym_coincidence_prob = (sym_pe_netnoise[true_success_stat]+sym_pe_netnoise[failure_stat]);

UndefVarError: UndefVarError: `sym_netnoise_opall` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

We can print out the symbolic expression:

In [23]:
latexstring(sym_true_success)

UndefVarError: UndefVarError: `latexstring` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

And evaluate it to compare it with the numerical calculation:

In [24]:
sym_true_success(netnoise_value, 0) # evaluate at e_n = netnoise_value, e_g = 0

UndefVarError: UndefVarError: `sym_true_success` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [25]:
pe_netnoise[true_success_stat]

0.6561

### Higher order expansions

Higher order expansions are also possible with this library, however the cost of the simulation grows exponentially in the order, due to the combinatorial explosion of trajectories that need to be followed. See the example below where we see the number of branches for the depolarization acting on all qubits.

In [26]:
for order in [0,1,2,3,4]
    branches = QuantumClifford.applynoise_branches(initial_state, netnoise, [1,2,3,4], max_order=order)
    println((order = order, nb_of_branches = length(branches), total_probability = sum(b[2] for b in branches)))
end

(order = 0, nb_of_branches = 1, total_probability = 0.6561)
(order = 1, nb_of_branches = 13, total_probability = 0.9476999999999999)
(order = 2, nb_of_branches = 67, total_probability = 0.9963000000000005)
(order = 3, nb_of_branches = 175, total_probability = 0.9999000000000001)
(order = 4, nb_of_branches = 256, total_probability = 1.0000000000000007)


Let us compute the whole circuits to the given order

In [27]:
@time sym_pe_netnoise2 = petrajectories(initial_state, [sym_netnoise_opall,circuit...],
                                  branch_weight=sym_unity, max_order=2);
@time sym_pe_netnoise3 = petrajectories(initial_state, [sym_netnoise_opall,circuit...],
                                  branch_weight=sym_unity, max_order=3);
@time sym_pe_netnoise4 = petrajectories(initial_state, [sym_netnoise_opall,circuit...],
                                  branch_weight=sym_unity, max_order=4);

UndefVarError: UndefVarError: `sym_netnoise_opall` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

### All of the results together

In [28]:
plot(title="Purified Fidelity vs Initial Fidelity\naccording to different calculation methods")

netnoise_pe_samples = 0.0:0.005:0.4
netnoise_mc_samples = 0.0:0.05:0.4
mc_trajectories = [50, 5000]

# No purification
exact_solution_no_purification = petrajectories(good_bell_state,
                                                [sym_netnoise_opall,nopurification_circuit...],
                                                branch_weight=sym_unity,
                                                max_order=100)
F0 = exact_solution_no_purification[true_success_stat].(netnoise_pe_samples,0)
plot!(F0,F0,color=:black,alpha=0.3,label=false)

# Symbolic perturbative expansions
for (order,sym_result) in enumerate(
        [sym_pe_netnoise,sym_pe_netnoise2,sym_pe_netnoise3,sym_pe_netnoise4])
    ts = sym_result[true_success_stat].(netnoise_pe_samples, 0)
    uf = sym_result[failure_stat].(netnoise_pe_samples, 0)
    Fout = ts ./ (ts .+ uf)
    plot!(F0, Fout, label="expansion to order=$(order)", lw=2)
end

# Monte Carlo approach
F0 = exact_solution_no_purification[true_success_stat].(netnoise_mc_samples,0)
for m in mc_trajectories
    Fout_mc = []
    for n in netnoise_mc_samples
        netnoise = UnbiasedUncorrelatedNoise(n)
        netnoise_opall = NoiseOpAll(netnoise)
        mc_netnoise = mctrajectories(initial_state, [netnoise_opall,circuit...],
                                     trajectories=m)
        push!(Fout_mc, mc_netnoise[true_success_stat]/(mc_netnoise[true_success_stat]+get(mc_netnoise,failure_stat,0)))
    end
    plot!(F0, Fout_mc, label="MC $(m) trajectories", line=false, marker=true)
end

plot!(legend=:outertopright, xlabel="Input Fidelity", ylabel="Output Fidelity")

UndefVarError: UndefVarError: `plot` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

## Local Operational Noise in Addition to the Network Noise

Up to here we considered imperfect initial states purified by perfect circuits. Now we will introduce imperfect gates as well, and study a few different regimes of the gate error model. We will add a small chance of depolarization to all of the gates and a chance for flipping a qubit to all measurements.

In [29]:
# A helper function to add noise to ops and lists of ops.
make_noisy(g::SparseGate, noise) = NoisyGate(g, UnbiasedUncorrelatedNoise(noise))
make_noisy(m::BellMeasurement, noise) = NoisyBellMeasurement(m, noise)
make_noisy(other_op, noise) = other_op
make_noisy(circuit::AbstractVector, noise) = [make_noisy(op, noise) for op in circuit];

In [30]:
sym_pe_allnoise = petrajectories(
    initial_state,
    [sym_netnoise_opall,make_noisy(circuit, e_gate)...],
    branch_weight=sym_unity)

sym_true_success = sym_pe_allnoise[true_success_stat]
sym_coincidence_prob = (sym_pe_allnoise[true_success_stat]+sym_pe_allnoise[failure_stat]);

UndefVarError: UndefVarError: `sym_netnoise_opall` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [31]:
latexstring(sym_true_success) # Not all that legible # TODO print this better

UndefVarError: UndefVarError: `latexstring` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [32]:
sym_allnoise_circuit = [sym_netnoise_opall,make_noisy(circuit, e_gate)...]

UndefVarError: UndefVarError: `sym_netnoise_opall` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [33]:
@time sym_pe_allnoise2 = petrajectories(
    initial_state,
    sym_allnoise_circuit,
    branch_weight=sym_unity,
    max_order=2);

UndefVarError: UndefVarError: `sym_unity` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [34]:
@time sym_pe_allnoise3 = petrajectories(
    initial_state,
    sym_allnoise_circuit,
    branch_weight=sym_unity,
    max_order=3);

UndefVarError: UndefVarError: `sym_unity` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [35]:
@time sym_pe_allnoise4 = petrajectories(
    initial_state,
    sym_allnoise_circuit,
    branch_weight=sym_unity,
    max_order=4);

UndefVarError: UndefVarError: `sym_unity` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

### All of the noisy circuit results together

In [36]:
plot(title="Purified Fidelity vs Initial Fidelity\nboth network and local noise\naccording to different calculation methods")

netnoise_pe_samples = 0.0:0.005:0.4
netnoise_mc_samples = 0.0:0.05:0.4
local_noise_samples = [0.0, 0.01, 0.05]
mc_trajectories = 10000

line_styles = [:dot, :dashdot, :dash, :solid]

# No purification
exact_solution_no_purification = petrajectories(good_bell_state,
                                                [sym_netnoise_opall,nopurification_circuit...],
                                                branch_weight=sym_unity,
                                                max_order=100)
F0 = exact_solution_no_purification[true_success_stat].(netnoise_pe_samples,0)
plot!(F0,F0,color=:black,alpha=0.3,label=false)

# Symbolic perturbative expansions
for (li, local_noise) in enumerate(local_noise_samples)
    for (order,sym_result) in enumerate(
            [sym_pe_allnoise,sym_pe_allnoise2,sym_pe_allnoise3,sym_pe_allnoise4])
        ts = sym_result[true_success_stat].(netnoise_pe_samples, local_noise)
        uf = sym_result[failure_stat].(netnoise_pe_samples, local_noise)
        Fout = ts ./ (ts .+ uf)
        plot!(F0, Fout,
            label = order==4 ? "gate error rate of $(local_noise)" : nothing,
            lw = order==4 ? 2 : 1.5,
            color=li, linestyle=line_styles[order],
        )
    end
end


# Monte Carlo approach
F0 = exact_solution_no_purification[true_success_stat].(netnoise_mc_samples,0)
for (li, local_noise) in enumerate(local_noise_samples)
    Fout_mc = []
    for n in netnoise_mc_samples
        netnoise = UnbiasedUncorrelatedNoise(n)
        netnoise_opall = NoiseOpAll(netnoise)
        c = [netnoise_opall,circuit...]
        c = make_noisy(c, local_noise)
        mc_netnoise = mctrajectories(initial_state, c,
                                     trajectories=mc_trajectories)
        push!(Fout_mc, mc_netnoise[true_success_stat]/(mc_netnoise[true_success_stat]+get(mc_netnoise,failure_stat,0)))
    end
    plot!(F0, Fout_mc, label=nothing, line=false, marker=true, color=li)
end

# Legend
for (li, style) in enumerate(line_styles)
    plot!([-1], [-1], linestyle = style, label = "perturbation to order $(li)", color = "black")
end
plot!([-1], [-1], line=false, marker=true, label = "10 000 MC trajectories", color = "black")
plot!(xlim=(0.39,1.01))
plot!(ylim=(0.35,1.03))
plot!(legend=:outertopright, xlabel="Input Fidelity", ylabel="Output Fidelity")

UndefVarError: UndefVarError: `plot` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

## Reproducibility information

In [37]:
versioninfo()
using Pkg
Pkg.status("QuantumClifford")

Julia Version 1.11.6
Commit 9615af0f269 (2025-07-09 12:58 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (arm64-apple-darwin24.0.0)
  CPU: 8 × Apple M3
  WORD_SIZE: 64
  LLVM: libLLVM-16.0.6 (ORCJIT, apple-m2)
Threads: 1 default, 0 interactive, 1 GC (on 4 virtual cores)
Status `~/Desktop/lab_work/QuantumClifford.jl/docs/Project.toml`
  [0525e862] QuantumClifford v0.10.0 `~/Desktop/lab_work/QuantumClifford.jl`
